<a href="https://colab.research.google.com/github/435wldms/essa/blob/main/20231010_%EA%B3%BC%EC%A0%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Facebook의 fasttext를 이용한 text classification

In [1]:
pip install fasttext

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.8/68.8 kB 2.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached pybind11-2.11.1-py3-none-any.whl (227 kB)
  Created wheel for fasttext: filename=fasttext-0.9.2-cp310-cp310-linux_x86_64.whl size=4199771 sha256=268a9daafa9fe424b0ce75eb1d6fbd54056d62c3e06a77ce63db783e3b9cacb4
  Stored in directory: /root/.cache/pip/wheels/a5/13/75/f811c84a8ab36eedbaef977a6a58a98990e8e0f1967f98f394
Successfully built fasttext


In [2]:
import fasttext
import pandas as pd
import numpy as np

train = pd.read_csv('train.csv')

file = open('fasttexttrain.txt', 'w+')
for i in train.index:
  line = '__label__'+str(train['author'][i])+''+train['text'][i]
  file.write(line+'\n')

In [ ]:
# Text Classification
# input             # training file path (required)
# model             # unsupervised fasttext model {cbow, skipgram} [skipgram]
# lr                # learning rate [0.05]
# dim               # size of word vectors [100]
# ws                # size of the context window [5]
# epoch             # number of epochs [5]
# minCount          # minimal number of word occurences [5]
# minn              # min length of char ngram [3]
# maxn              # max length of char ngram [6]
# neg               # number of negatives sampled [5]
# wordNgrams        # max length of word ngram [1]
# loss              # loss function {ns, hs, softmax, ova} [ns]
# bucket            # number of buckets [2000000]
# thread            # number of threads [number of cpus]
# lrUpdateRate      # change the rate of updates for the learning rate [100]
# t                 # sampling threshold [0.0001]
# verbose           # verbose [2]

In [3]:
text_clf_model = fasttext.train_supervised('fasttexttrain.txt', epoch=30, minCount=2, maxn=10, verbose=0)
print(text_clf_model.words)
print(text_clf_model.labels)

['the', 'and', 'to', 'of', '</s>', 'a', 'I', 'in', 'was', 'that', 'he', 'his', 'you', 'with', 'had', 'it', 'for', 'as', 'odin', 'at', 'not', 'my', 'have', 'is', 'be', 'her', 'said', 'on', 'she', 'but', 'odin,', 'all', 'from', 'me', 'by', 'him', 'were', 'which', 'so', 'this', 'been', 'would', 'an', 'your', 'are', 'very', 'no', 'one', 'could', 'what', 'we', 'they', 'Mr.', 'will', 'if', 'or', 'upon', 'He', 'there', 'when', 'out', 'The', 'do', 'who', 'up', 'some', 'am', 'odin.', 'more', 'into', 'their', 'about', 'only', 'should', 'little', 'must', 'than', 'like', 'any', 'It', 'know', 'such', 'did', 'man', 'has', 'But', 'see', 'them', 'can', 'our', 'never', 'much', 'come', 'before', 'down', 'may', 'think', 'how', 'And', 'made', 'it,', 'time', 'me,', 'came', 'now', 'great', 'two', 'good', 'might', 'old', 'You', 'She', 'over', 'shall', 'after', 'own', 'cried', 'though', 'him,', 'go', 'it.', 'say', 'without', 'even', 'other', 'you,', 'looked', 'then', 'thought', 'every', 'last', 'first', 'bein

In [4]:
reuslt = text_clf_model.predict("He was almost choking. There was so much, so much he wanted to say, but strange exclamations were all that came from his lips. The Pole gazed fixedly at him, at the bundle of notes in his hand; looked at odin, and was in evident perplexity.", k=5)
print(reuslt)

(('__label__3odin', '__label__3The', '__label__3He', '__label__0The', '__label__0I'), array([0.12350754, 0.06864063, 0.06747385, 0.03694613, 0.0277812 ]))


In [5]:
test = pd.read_csv('test_x.csv')
submission = pd.read_csv('sample_submission.csv', index_col = False)

In [6]:
for i in test.index:
  lable, proba = text_clf_model.predict(test['text'][i],k=5)
  for la, pr in zip(lable,proba):
    if '__label__0'==la:
      submission.loc[i,'0'] = pr
    elif '__label__1'==la:
      submission.loc[i,'1'] = pr
    elif '__label__2'==la:
      submission.loc[i,'2'] = pr
    elif '__label__3'==la:
      submission.loc[i,'3'] = pr
    elif '__label__4'==la:
      submission.loc[i,'4'] = pr

submission.to_csv('result5_fasttext.csv', index=False)
print('end')

end


# NLP 전처리

* NLP 전처리는 정해진 정답이 없는데, 주로 모델의 입력인 단어, 문장, 문서의 vector를 만들기 전에 진행됨
* 전체 process: 데이터를 불러온 후 기사를 눈으로 확인하며 특수문자, 불용어, 문장 구조에 대한 감 잡기 -> 불용어를 설정하고 리스트에 저장 -> 불용어 이외의 특수문자들을 제거 -> 형태소 분석을 통해 문장을 형태소 단위의 토큰으로 분리 -> 형태소 단위의 토큰들을 기반으로 리스트에 저장된 불용어 제거

**1. 형태소 분석(Stemming)**

* 단어나 문장의 언어적 속성을 파악하는 것으로 형태소 단위로 의미있는 단어를 가져가고 싶거나 태깅을 통해 형용사, 명사를 추출하고 싶을 때 사용
* 문장을 띄어쓰기 단위로 분류하여 vectorization을 하게 되면 '데이콘'이라는 같은 의미의 토큰 세 개가 서로 다른 벡터를 갖게 됨 -> 모델이 같은 의미를 가진 단어임에도 형태소에 따라 다른 단어로 이해함
* 모델링보다 성능에 더 중요한 영향을 미칠 수 있음

**1-1 Kkma()**

In [7]:
pip install konlpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 60.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.3/465.3 kB 34.9 MB/s eta 0:00:00


In [8]:
from konlpy.tag import Kkma
kkma = Kkma()

sentence = '데이콘에서 다양한 컴피티션을 즐기면서 실력있는 데이터 분석가로 성장하세요!!.'

print("형태소 단위로 문장 분리")
print("----------------------")
print(kkma.morphs(sentence))

print("문장에서 명사 추출")
print("----------------------")
print(kkma.nouns(sentence))

print("품사 태킹(PoS)")
print("----------------------")
print(kkma.pos(sentence))

형태소 단위로 문장 분리
----------------------
['데이', '콘', '에서', '다양', '하', 'ㄴ', '컴피티션', '을', '즐기', '면서', '실력', '있', '는', '데이터', '분석가', '로', '성장', '하', '세요', '!!', '.']
문장에서 명사 추출
----------------------
['데이', '데이콘', '콘', '다양', '컴피티션', '실력', '데이터', '분석가', '성장']
품사 태킹(PoS)
----------------------
[('데이', 'NNG'), ('콘', 'NNG'), ('에서', 'JKM'), ('다양', 'NNG'), ('하', 'XSV'), ('ㄴ', 'ETD'), ('컴피티션', 'UN'), ('을', 'JKO'), ('즐기', 'VV'), ('면서', 'ECE'), ('실력', 'NNG'), ('있', 'VV'), ('는', 'ETD'), ('데이터', 'NNG'), ('분석가', 'NNG'), ('로', 'JKM'), ('성장', 'NNG'), ('하', 'XSV'), ('세요', 'EFN'), ('!!', 'SW'), ('.', 'SF')]


**1-2 Okt()**

In [9]:
from konlpy.tag import Okt
Okt = Okt()

sentence = '데이콘에서 다양한 컴피티션을 즐기면서 실력있는 데이터 분석가로 성장하세요!!.'

print("형태소 단위로 문장 분리")
print("----------------------")
print(Okt.morphs(sentence))

print("문장에서 명사 추출")
print("----------------------")
print(Okt.nouns(sentence))

print("품사 태킹(PoS)")
print("----------------------")
print(Okt.pos(sentence))

형태소 단위로 문장 분리
----------------------
['데', '이콘', '에서', '다양한', '컴피티션', '을', '즐기면서', '실력', '있는', '데이터', '분석', '가로', '성장하세요', '!!.']
문장에서 명사 추출
----------------------
['데', '이콘', '컴피티션', '실력', '데이터', '분석', '가로']
품사 태킹(PoS)
----------------------
[('데', 'Noun'), ('이콘', 'Noun'), ('에서', 'Josa'), ('다양한', 'Adjective'), ('컴피티션', 'Noun'), ('을', 'Josa'), ('즐기면서', 'Verb'), ('실력', 'Noun'), ('있는', 'Adjective'), ('데이터', 'Noun'), ('분석', 'Noun'), ('가로', 'Noun'), ('성장하세요', 'Adjective'), ('!!.', 'Punctuation')]


이 외에도 다양한 형태소 분석기 존재
* Mecab: 굉장히 속도가 빠르면서도 좋은 분석 결과를 보여준다.
* Komoran: 댓글과 같이 정제되지 않은 글에 대해서 먼저 사용해보면 좋다.(오탈자를 어느정도 고려해준다.)
* Kkma: 분석 시간이 오래걸리기 때문에 잘 이용하지 않게 된다.
* Okt: 품사 태깅 결과를 Noun, Verb등 알아보기 쉽게 반환해준다.
* khaiii: 카카오에서 가장 최근에 공개한 분석기, 성능이 좋다고 알려져 있으며 다양한 실험이 필요하다.

**2. 표제어 추출(Lemmatization)**

* 단어의 본 모습을 찾아주는 과정(stemming과 유사)

In [10]:
from konlpy.tag import Kkma
kkma = Kkma()

sentence = '성장했었다.'

print('품사 태킹(PoS)')
print("----------------------")
print(kkma.pos(sentence))

품사 태킹(PoS)
----------------------
[('성장', 'NNG'), ('하', 'XSV'), ('었', 'EPT'), ('었', 'EPT'), ('다', 'EFN'), ('.', 'SF')]


In [11]:
sentence = '성장하였었다.'

print("품사 태킹(PoS)")
print("----------------------")
print(kkma.pos(sentence))

품사 태킹(PoS)
----------------------
[('성장', 'NNG'), ('하', 'XSV'), ('였', 'EPT'), ('었', 'EPT'), ('다', 'EFN'), ('.', 'SF')]


**3. 불용어 제거**

* 문장에서 큰 의미가 없다고 생각되는 단어
* Example: "이번에 새롭게 개봉한 영화의 배우들은 모두 훌륭한 연기력과 아름다운 목소리를 갖고 있어!!"  에서 '훌륭한', '아름다운' 등의 형용사들은 중요하지 않게 작용함 -> 경우에 따라서는 중요할수도 있기 때문에 상황에 따라 불용어는 달라짐

In [14]:
from konlpy.tag import Okt
import re
tokenizer = Okt()

def text_preprocessing(text, tokenizer):

  stopwords =['을', '를', '이', '가', '은', '는']
  txt = re.sub('[^가-힣a-z]', '', text)  # 영어 소문자와 한글을 제외한 모든 문자를 제거
  token = tokenizer.morphs(txt)
  token = [t for t in token if t not in stopwords] # 형태소 분석기를 거쳐 나온 결과들 중 stopwords 리스트에 포함되지 않는 토큰만 token 리스트에 반환

  return token

ex_text = "이번에 새롭게 개봉한 영화의 배우들은 모두 훌륭한 연기력과 아름다운 목소리를 갖고 있어!!"
example_pre = text_preprocessing(ex_text, tokenizer)

In [15]:
print(example_pre)

['이번', '에', '새롭게', '개봉', '한', '영화', '의', '배우', '들', '모두', '훌륭한', '연기력', '과', '아름다운', '목소리', '갖고있어']


**4. 대회 적용**

In [ ]:
def text_preprocessing(text_list):

    stopwords = ['을', '를', '이', '가', '은', '는', 'null'] #불용어 설정
    tokenizer = Okt() #형태소 분석기
    token_list = []

    for text in text_list:
        txt = re.sub('[^가-힣a-z]', ' ', text) #한글과 영어 소문자만 남기고 다른 글자 모두 제거
        token = tokenizer.morphs(txt) #형태소 분석
        token = [t for t in token if t not in stopwords or type(t) != float] #형태소 분석 결과 중 stopwords에 해당하지 않는 것만 추출
        token_list.append(token)

    return token_list, tokenizer

# 형태소 분석기를 따로 저장한 이유는 후에 test 데이터 전처리를 진행할 때 이용해야 하기 떄문
train['new_article'], okt = text_preprocessing(train['content'])

# NLP Vectorization

* nlp를 컴퓨터가 이해할 수 있게 수치로 바꾸는 것

In [17]:
# 간단한 전처리
from konlpy.tag import Okt
import re

Okt = Okt()

sentences = ['자연어 처리는 정말 정말 즐거워.', '즐거운 자연어 처리 다같이 해보자.']
tokens = []

for sentence in sentences:
  sentence = re.sub('[^가-힣a-z]', ' ', sentence)
  token = (Okt.morphs(sentence))
  tokens.append(''.join(token))

print('형태소 단위로 문장 분리')
print("----------------------")
print(tokens)

형태소 단위로 문장 분리
----------------------
['자연어처리는정말정말즐거워', '즐거운자연어처리다같이해보자']


**1. One Hot Encoding**

* 해당 단어가 존재하면 1, 그렇지 않으면 0으로 표시
* keras를 이용하면 쉽게 구현이 가능
* texts_to_sequences를 활용해 각 토큰에 고유한 정수를 부여한 후 to_categorical을 활용하면 구현 가능

In [18]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical

t = Tokenizer()
t.fit_on_texts(tokens)
print('각 토큰에게 고유의 정수 부여')
print("----------------------")
print(t.word_index)
print(" ")

s1 = t.texts_to_sequences(tokens)[0]
print("부여된 정수로 표시된 문장1")
print("----------------------")
print(s1)
print(" ")

s2=t.texts_to_sequences(tokens)[1]
print("부여된 정수로 표시된 문장2")
print("----------------------")
print(s2)
print(" ")

s1_one_hot = to_categorical(s1)
print("문장1의 one-hot-encoding")
print("----------------------")
print(s1_one_hot)
print(" ")

s2_one_hot = to_categorical(s2)
print("문장2의 one-hot-encoding")
print("----------------------")
print(s2_one_hot)

각 토큰에게 고유의 정수 부여
----------------------
{'자연어처리는정말정말즐거워': 1, '즐거운자연어처리다같이해보자': 2}
 
부여된 정수로 표시된 문장1
----------------------
[1]
 
부여된 정수로 표시된 문장2
----------------------
[2]
 
문장1의 one-hot-encoding
----------------------
[[0. 1.]]
 
문장2의 one-hot-encoding
----------------------
[[0. 0. 1.]]


-> but 이 방식은 vocabulary 크기가 커짐에 따라 많은 공간을 차지하게 되고 벡터가 sparse해지기 때문에 모델에게 좋은 특성을 알려주지 못하는 경우가 많음

**2. Count Vectorization**
* 각 문장이 가지는 토큰의 count를 기반으로 문장을 vectorization함

In [22]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
vectors = vectorizer.fit_transform(tokens)

print(vectorizer.get_feature_names_out())
print(vectors.toarray())

['자연어처리는정말정말즐거워' '즐거운자연어처리다같이해보자']
[[1 0]
 [0 1]]


-> 벡터화 결과, 각 문장은 vocabulary 인덱스 기준으로 카운트를 정수로 표시 (한 글자는 자동으로 제거)
-> 즐거운, 즐거워는 같은 의미이지만 앞의 전처리에서 okt가 구분해주지 못했기 때문에 다른 토큰으로 분리됨

**3. Tfldf**
* 단어가 몇 번 등장했는지 + 어떤 단어가 언급된 문서의 수 가 적으면 그 단어는 문서 분류에 중요한 단어 -> 로 해석

In [20]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(min_df =0)
tfidf_vectorizer = tfidf.fit_transform(tokens)

# tf-idf dictionary
tfidf_dict = tfidf.get_feature_names_out()
print(tfidf_dict)
print(tfidf_vectorizer.toarray())

['자연어처리는정말정말즐거워' '즐거운자연어처리다같이해보자']
[[1. 0.]
 [0. 1.]]


-> 각 단어에 대한 가중치가 다르게 표시됨

**4. Padding**

* dataframe 형식에서는 row별 동일한 column 수를 가져야하는데, nlp에서는 row(문장)별 column(토큰)의 개수가 다른 문제가 생김
* 기본적으로 길이를 동일하게 맞춰주어야 함 -> 부족한 길이에 0을 채움

**5. 대회 적용**
* 특별한 vectorization 기법을 사용하지 않고 texts_to_sequences를 통한 정수화만 진행하여도 높은 정확도를 얻을 수 있음

In [23]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

def text2sequence(train_text, max_len=100):

    tokenizer = Tokenizer() #keras의 vectorizing 함수 호출
    tokenizer.fit_on_texts(train_text) #train 문장에 fit
    train_X_seq = tokenizer.texts_to_sequences(train_text) #각 토큰들에 정수 부여
    vocab_size = len(tokenizer.word_index) + 1 #모델에 알려줄 vocabulary의 크기 계산
    print('vocab_size : ', vocab_size)
    X_train = pad_sequences(train_X_seq, maxlen = max_len) #설정한 문장의 최대 길이만큼 padding

    return X_train, vocab_size, tokenizer

train_X, vocab_size, vectorizer = text2sequence(train['text'], max_len = 100)

vocab_size :  42331
